# Zero-shot learning et few-shot learning avec un LLM

Dans ce TP, nous allons expérimenter les concepts de zero-shot et few-shot learning qui deviennent de plus en plus importants pour le changement de paradigme en Intelligence Artificielle basé sur l'apprentissage auto-supervisé.

Que signifient zero-shot et few-shot learning ?

- En **zero-shot learning**, on utilise un modèle pré-entraîné (= un modèle de fondation) pour une classe d'objets ou d'inputs qu'il n'a jamais vu. Exemple : on demande à un modèle entraîné à détecter les voitures et les humains dans une image de détecter tous les bâtiments sans jamais lui avoir donné de données annotés de bâtiments en entraînement.

- En **few-shot learning**, on donne par contre quelques exemples (seulement) à ce modèle pré-entraîné.

Il y a de nombreuses façons de faire du zero-shot/few-shot learning. Tout dépend du type de données que l'on considère (texte, image, vidéo ou autre) et de la structure du modèle de fondation que l'on considère.

En pratique, lorsqu'il possible de remplir une tâche en mode zero-shot learning, c'est la première chose que l'on teste. Selon la qualité du résultat, on teste ensuite le few-shot learning, avant d'éventuellement se lancer dans du fine-tuning ou des méthodes plus complexes d'adaptation de modèle.

Quelques exemples de stratégies de zero/few-shot learning pour les LLMs :
- **zero/few-shot prompting** : on ajoute des exemples dans le contexte du prompt ;

- Faire du fine-tuning avec quelques exemples ;

- Le Reinforcement Learning from Human Feedback (RLHF) est une méthode de few-shot learning.


In [ ]:
!pip install pandas

## 1) Exemple : zero-shot prompting pour classification

Ici nous allons donner un exemple d'utilisation de zero-shot prompting pour classifier des critiques (*reviews*) de films.

Le zero-shot ou few-shot prompting est souvent aussi appelé "in-context learning", mais dans un contexte assez spéficique aux LLMs.


On commence par écrire la fonction qui va nous permettre de classifier via un prompt si une review est positive ou négative. Nous nous basons largement sur les éléments de code vus dans le Jupyter notebook précédent.

In [ ]:
from typing import Dict

from openai import OpenAI

from credentials.keys import OPENAI_API_KEY

client = OpenAI(api_key=OPENAI_API_KEY)


def send_prompt_with_context(
        messages: list[Dict],
        model: str = 'gpt-4o-mini',
        temperature: float = 0.1) -> str:
    '''
    Envoi d'un message au modèle.
    '''
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.1,
        max_tokens=200
    )
    return resp.choices[0].message.content

Essayons sur un exemple simple :

In [ ]:
question = "Is the movie associated with this review positive or negative ?"
review = "Very bad movie"

system_msg = f"""
You are an assistant that classifies reviews according to their sentiment. \
Respond in json format with the key: gpt_sentiment.\
The value for gpt_sentiment should only be either pos or neg without punctuation: pos if the review is positive, neg otherwise.\
"""

messages = [
    {"role": "system", "content": system_msg},
    {"role": "system", "content": f"Consider the following review {review}"},
    {"role": "user", "content": f"Question: {question}"}
]

other_answer = send_prompt_with_context(messages)
other_answer

'{\n  "gpt_sentiment": "neg"\n}'

La réponse étant donnée en format JSON, il faut l'interpréter via la librarie `json` :

In [ ]:
import json
data = json.loads(other_answer)

Téléchargeons maintenant le dataset [**Large Movie Review**](https://ai.stanford.edu/~amaas/data/sentiment/).

Il est structuré de la façon suivante :

```
aclImdb/\
├── test/ \
│   ├── neg/\
│   └── pos/\
├── train/\
│   ├── neg/\
│   ├── pos/\
│   └── unsup/\
└── imdb.vocab
```

Le folder `unsup/` dans `train/` correspond seulement au cas où nous voudrions faire de l'entraînement non-supervisé.

In [ ]:
import os

if not os.path.exists('aclImdb'):
    !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
    !tar -xzf aclImdb_v1.tar.gz

--2024-10-13 11:33:13--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  11.8MB/s    in 11s     

2024-10-13 11:33:25 (7.23 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



On charge le dataset :

In [ ]:
import os
import pandas as pd

def load_imdb_dataset(dataset_path: str, nbr_max_per_class: int = 20):
    '''
    Inputs:
    -------
        nbr_max_per_class (int) : nombre maximum de reviews à charger pour
            chaque classe (positive et négative). On ne veut pas pour cet
            exercice considérer toutes les reviews pour éviter de payer
            trop cher en OpenAI API :-)
    '''
    data = {'review': [], 'sentiment': []}

    for sentiment in ['pos', 'neg']:
        path = os.path.join(dataset_path, sentiment)
        for i, file_name in enumerate(os.listdir(path)):
            with open(os.path.join(path, file_name), 'r', encoding='utf-8') as file:
                data['review'].append(file.read())
                data['sentiment'].append(sentiment)
            if i >= nbr_max_per_class - 1:
                break

    return pd.DataFrame(data)

# Load training and testing datasets
train_dataset = load_imdb_dataset('aclImdb/train')
test_dataset = load_imdb_dataset('aclImdb/test')

print(f'Loaded {len(train_dataset)} reviews.')
print(train_dataset.head())

Loaded 40 reviews.
                                              review sentiment
0  Yes, it's another great magical Muppet's movie...       pos
1  This movie is outrageous, funny, ribald, sophi...       pos
2  ...would probably be the best word to describe...       pos
3  Dog Bite Dog isn't going to be for everyone, b...       pos
4  WWE Armageddon, December 17, 2006 -- Live from...       pos


Testons le prompt sur un exemple de review

In [ ]:
review = train_dataset['review'][1]
label = train_dataset['sentiment'][1]

print(f'This is the review: \n {review}')
print(f'It is labeled as {label} in the dataset.')

This is the review: 
 This movie is outrageous, funny, ribald, sophisticated & hits the bullseye where 99 % of Hollywood movies don't even make the target. Paul Bartel should be recognized as one of the great directors of this or any era. He's the American Renoir & Bunuel _ combined!!! Glad I have the videodisc.
It is labeled as pos in the dataset.


Testons maintenant un prompt pour faire de la zero-shot classification.

In [ ]:
question = "Is the movie associated with this review positive or negative ?"

system_msg = f"""
You are an assistant that classifies reviews according to their sentiment. \
Respond in json format with the key: gpt_sentiment, for example: '{{\n  "gpt_sentiment": "pos"\n}}'.\
The value for gpt_sentiment should only be either pos or neg without punctuation: pos if the review is positive, neg otherwise.\
"""

messages = [
    {"role": "system", "content": system_msg},
    {"role": "system", "content": f"Consider the following review {review}"},
    {"role": "user", "content": f"Question: {question}"}
]

other_answer = send_prompt_with_context(messages)

# On vérifie le format du prompt.
wrong_format = False
try:
    data = json.loads(other_answer)
except:
    print('Could not load the data to json.')

if 'gpt_sentiment' not in data.keys():
    wrong_format = True
elif data['gpt_sentiment'] not in ['pos', 'neg']:
    wrong_format = True

if wrong_format:
    print('GPT did not provide the right format for the answer')
else:
    print(f'This review has been classified as {data['gpt_sentiment']} by GPT and was {label} in the dataset')

{'gpt_sentiment': 'pos'}
This review has been classified as pos by GPT and was pos in the dataset


**Exercice** : Calculer la performance de cette approche. Calculer le nombre de cas où le format de la réponse n'est pas correct.

On calcule l'accuracy sur tout le dataset

<class 'pandas.core.frame.DataFrame'>


100%|██████████| 5/5 [00:04<00:00,  1.12it/s]

0.0% of examples are wrongly formatted.
This method has 80.0% accuracy.


0.0% of examples are wrongly formatted.
This method has 10.0% accuracy.


## TP) Exemple de few-shot prompting pour classification

Nous allons maintenant effectuer la même chose, sauf que nous allons faire du few-shot prompting. Le TP est très peu encadré. L'objectif est que vous adaptiez le code précédent pour faire du few-shot prompting.

Quelques étapes suggérées :

- Formatter un objet `Message` pour pouvoir inclure différents exemples de review.

- Calculer la performance de cette approche.

D'autres aspects à expérimenter:

- Quel est l'effet d'augmenter la température sur les performances ?

- Quel est l'effet du nombre d'exemples en few-shot learning sur les performances?

,review,sentiment
0,"Yes, it's another great magical Muppet's movie...",pos
1,"This movie is outrageous, funny, ribald, sophi...",pos
2,...would probably be the best word to describe...,pos
3,"Dog Bite Dog isn't going to be for everyone, b...",pos
4,"WWE Armageddon, December 17, 2006 -- Live from...",pos


## TP Bonus : Pour aller plus loin

Une limitation très importante de cette approche est que les call APIs au modèle `chat.completion` sont assez lents, ce qui prend donc beaucoup de temps pour arriver à faire une bonne classification.

On pourrait aussi simplement faire des embedding des différentes reviews, puis entrainer un classifieur basique (classification binaire).

Cette approche est-elle plus rapide ? Ses performances sont-elles différentes ?